
# 作業目標:
    
    了解keras 模型架構


# 作業重點:
    
    請修改input shape: (Conv2D(64, (3, 3))的設定, 新增一層 Dense 並觀看 model.summary 的輸出


In [1]:
# 載入必須使用的 Library
import keras
from keras.datasets import cifar10
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
num_classes = 10
batch_size = 256
epochs = 50

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
unique_labels = np.unique(y_train)
num_classes = len(unique_labels)
print(len(unique_labels), ' labels')

# Normlized Inputs
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
10  labels


In [3]:
# build our CNN model, 多加幾層
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# (CONV => RELU) * 2 => POOL
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# first (and only) set of FC => RELU layers
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 10, 10, 64)        0         
__________

In [4]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_history = model.fit(x=x_train, y=y_train,
                                    batch_size=batch_size,
                                    epochs=epochs, shuffle=True,
                                    verbose=1,
                                    validation_data=[x_test, y_test],
                                    callbacks=[es,mc])


Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 6s 112us/step - loss: 1.9526 - acc: 0.3838 - val_loss: 1.7411 - val_acc: 0.4375

Epoch 00001: val_acc improved from -inf to 0.43750, saving model to best_model.h5
Epoch 2/50
50000/50000 [==============================] - 4s 86us/step - loss: 1.3411 - acc: 0.5309 - val_loss: 1.3489 - val_acc: 0.5247

Epoch 00002: val_acc improved from 0.43750 to 0.52470, saving model to best_model.h5
Epoch 3/50
50000/50000 [==============================] - 4s 74us/step - loss: 1.1191 - acc: 0.6037 - val_loss: 1.3465 - val_acc: 0.5441

Epoch 00003: val_acc improved from 0.52470 to 0.54410, saving model to best_model.h5
Epoch 4/50
50000/50000 [==============================] - 4s 80us/step - loss: 0.9959 - acc: 0.6455 - val_loss: 0.9796 - val_acc: 0.6526

Epoch 00004: val_acc improved from 0.54410 to 0.65260, saving model to best_model.h5
Epoch 5/50
50000/50000 [==============================] - 4s